In [47]:
%cd ~/morphological-inflection/preprocessing/hierarchical_schema/src/
!ls

/usr/local/lib/python3.10/dist-packages/IPython/core/magics/osm.py:393: UserWarning: This is now an optional IPython functionality, using bookmarks requires you to install the `pickleshare` library.
  bkms = self.shell.db.get('bookmarks', {})
/usr/local/lib/python3.10/dist-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


/dss/dsshome1/03/ge87wod2/morphological-inflection/preprocessing/hierarchical_schema/src
hierarchical_features.ipynb


In [57]:
# split and sample from the paradigms according to the SIGMORPHON–UniMorph 2023 Shared Task 0 Paper
def get_lemma(line):
    lemma = line.split("\t")[0]
    return lemma

from random import sample,shuffle
from math import floor
from itertools import groupby
path_data = "../data/"
LANGS = ["ces","dsb","slk"]
LANGS = ["ote","pol","csb"]

for lang in LANGS:
    with open (path_data + lang,"r") as data_file, open (path_data + lang + ".trn","w") as train_file, open (path_data + lang + ".dev","w") as dev_file,open (path_data + lang + ".tst","w") as test_file:
        data_lines = [line.strip() for line in data_file.readlines()]
        tables = [(k, list(g)) for k, g in groupby(data_lines, get_lemma) if k != ""]
        if lang != "csb":
            for n in (500,1000,2000,3000):
                if len (tables) < n: n = len (tables)
                sampled_tables = sample(tables,n)
                size = len(sampled_tables)
                datasets = [ sampled_tables[:floor(0.8*size)], sampled_tables[floor(0.8*size):floor(0.9*size)], sampled_tables[floor(0.9*size):] ]

                train_forms = [form for lemma,table in datasets[0] for form in table]
                dev_forms = [form for lemma,table in datasets[1] for form in table]
                test_forms = [form for lemma,table in datasets[2] for form in table]

                if len(dev_forms) < 1000 or len(test_forms) < 1000 or len(train_forms) < 10000:
                    print(f"more tables than {n} needed for lang: {lang} ",len(dev_forms),len(test_forms) ,len(train_forms))
                else:
                    print(f"{lang} needed {n} tables.")
                    break
                    

            # sampling and retaining order of the forms
            indices_train = sorted(sample(range(len(train_forms)), 10000))
            indices_dev =  sorted(sample(range(len(dev_forms)), 1000))
            indices_test = sorted( sample(range(len(test_forms)), 1000))

            train_samples = [train_forms[i] for i in indices_train]
            dev_samples = [dev_forms[i] for i in indices_dev]
            test_samples = [test_forms[i] for i in indices_test]
            
           
            train_txt = "\n".join(train_samples) + "\n"
            dev_txt = "\n".join(dev_samples) + "\n"
            test_txt = "\n".join(test_samples) + "\n"

            train_file.write(train_txt)
            dev_file.write(dev_txt)
            test_file.write(test_txt)
        elif lang == "csb":
            # shuffle lemmas
            shuffle(tables) # lemmas should be randomly ordered
            test_forms = [form for lemma,table in tables for form in table]
            length = len([(k, list(g)) for k, g in groupby(test_forms, get_lemma)])
            print(f"number of lemmas in csb testfile: ",length)
            test_txt = "\n".join(test_forms) + "\n"
            test_file.write(test_txt)


            
        


more tables than 500 needed for lang: ote  798 744 6426
ote needed 1000 tables.
more tables than 500 needed for lang: pol  851 772 7001
pol needed 1000 tables.
number of lemmas in csb testfile:  37


In [70]:
# unique tags    
tags = []
path_data = "../data/"
LANGS = ["ote","pol","csb","ces","dsb","slk"]
datasets = ["trn","dev","tst"]
for lang in LANGS:
    if lang == "csb": datasets = ["tst"]
    else: datasets = ["trn","dev","tst"]
    for dataset in datasets:
        with open (path_data + lang + "."+dataset,"r") as data_file:
            print(data_file)
            data_lines = data_file.readlines()
            for line in data_lines:
                line = line.strip()
                if line == "": continue
                lemma, target, features = line.split("\t",2)
                features = features.replace("\t",";") # features are in last or two last columns
                features = features.split(";")
                for feature in features: 
                    if feature not in tags: 
                        tags += [feature]   
print(tags)



<_io.TextIOWrapper name='../data/ote.trn' mode='r' encoding='UTF-8'>
<_io.TextIOWrapper name='../data/ote.dev' mode='r' encoding='UTF-8'>
<_io.TextIOWrapper name='../data/ote.tst' mode='r' encoding='UTF-8'>
<_io.TextIOWrapper name='../data/pol.trn' mode='r' encoding='UTF-8'>
<_io.TextIOWrapper name='../data/pol.dev' mode='r' encoding='UTF-8'>
<_io.TextIOWrapper name='../data/pol.tst' mode='r' encoding='UTF-8'>
<_io.TextIOWrapper name='../data/csb.tst' mode='r' encoding='UTF-8'>
<_io.TextIOWrapper name='../data/ces.trn' mode='r' encoding='UTF-8'>
<_io.TextIOWrapper name='../data/ces.dev' mode='r' encoding='UTF-8'>
<_io.TextIOWrapper name='../data/ces.tst' mode='r' encoding='UTF-8'>
<_io.TextIOWrapper name='../data/dsb.trn' mode='r' encoding='UTF-8'>
<_io.TextIOWrapper name='../data/dsb.dev' mode='r' encoding='UTF-8'>
<_io.TextIOWrapper name='../data/dsb.tst' mode='r' encoding='UTF-8'>
<_io.TextIOWrapper name='../data/slk.trn' mode='r' encoding='UTF-8'>
<_io.TextIOWrapper name='../data/s

In [71]:
# classify each feature, sort and rearrange into hierarchical schema
features_in_ote_pol_csb_new = ['V', 'IPFV', 'SG', '1', 'PRS', 'PST', '2', '3', 'PFV', 'PRF', 'IRR', 'MASC', 'N', 'ESS', 'PL', 'INS', 'NOM', 'ACC', 'DAT', 'GEN', 'VOC', 'ADJ', 'FEM', 'NEUT', 'HUM', 'INAN', 'ANIM', 'FUT', 'IMP', 'V.PTCP', 'ACT', 'V.CVB', 'COND', 'PASS', 'V.MSDR', 'NFIN']
features = ['V', 'IPFV', 'SG', '2', 'PRS', '3', 'PST', 'PFV', 'PRF', '1', 'IRR', 'N', 'ESS', 'PL', 'INS', 'NOM', 'GEN', 'DAT', 'ACC', 'VOC', 'ADJ', 'FEM', 'NEUT', 'MASC', 'ANIM', 'HUM', 'INAN', 'IMP', 'FUT', 'V.MSDR', 'V.PTCP', 'PASS', 'COND', 'V.CVB', 'ACT', 'NFIN', 'ADV', 'CMPR', 'SPRL', 'DU', 'IND', 'NEG']
features_new = ['V', 'IPFV', 'SG', '1', 'PRS', '2', '3', 'PST', 'PFV', 'PRF', 'IRR', 'ADJ', 'FEM', 'ACC', 'INS', 'NOM', 'PL', 'NEUT', 'MASC', 'ANIM', 'GEN', 'HUM', 'DAT', 'ESS', 'N', 'VOC', 'COND', 'INAN', 'FUT', 'IMP', 'NFIN', 'V.PTCP', 'ACT', 'V.MSDR', 'PASS', 'V.CVB', 'ADV', 'CMPR', 'SPRL', 'DU', 'IND', 'NEG']
for f in features_new:
    if f not in features:
        print("add feature ", f)


In [72]:

# soruce https://aclanthology.org/P15-2111.pdf

features_per_dimension = {
    "Aktionsart": ["ACCMP", "ACH", "ACTY", "ATEL", "DUR", "DYN", "PCT", "SEMEL", "STAT", "TEL"],
    'Animacy': ["ANIM", "HUM", "INAN", "NHUM"],
    'Aspect': ["HAB", "IPFV", "ITER", "PFV", "PRF", "PROG", "PROSP"],
    "Case": ["ABL", "ABS", "ACC", "ALL", "ANTE", "APPRX", "APUD", "AT", "AVR", "BEN", "CIRC", "COM", "COMPV", "DAT", "EQU",
"ERG", "ESS", "FRML", "GEN", "INS", "IN", "INTER", "NOM", "NOMS", "ON", "ONHR", "ONVR", "POST", "PRIV", "PROL",
"PROPR", "PROX", "PRP", "PRT", "REM", "SUB", "TERM", "VERS", "VOC"],
"Comparison": ["AB", "CMPR", "EQT", "RL", "SPRL"],
"Definiteness": ["DEF", "INDEF", "NSPEC", "SPEC"],
"Deixis": ["ABV", "BEL", "DIST", "EVEN", "MED", "NVIS", "PROX", "REF1", "REF2", "REM", "VIS" ],
"Evidentiality": ["ASSUM", "AUD", "DRCT", "FH", "HRSY", "INFER", "NFH" , "NVSEN", "QUOT", "RPRT", "SEN"],
"Finiteness": ["FIN", "NFIN" ],
"Gender+" :["BANTU1-23", "FEM", "MASC", "NAKH1-8", "NEUT" ],
"Info. Structure": ["FOC", "TOP" ],
"Interrogativity": ["DECL", "INT" ],
"Mood": ["ADM", "AUNPRP", "AUPRP", "COND", "DEB", "IMP", "IND", "INTEN", "IRR", "LKLY", "OBLIG", "OPT",
"PERM", "POT", "PURP", "REAL", "SBJV", "SIM"],
"Number": ["DU", "GPAUC", "GRPL", "INVN", "PAUC", "PL", "SG", "TRI"],
"Parts of Speech": ["ADJ", "ADP", "ADV", "ART", "AUX", "CLF", "COMP", "CONJ", "DET", "INTJ", "N", "NUM", "PART", "PRO",
"V", "V.CVB", "V.MSDR", "V.PTCP"],
"Person": ["0", "1", "2", "3", "4", "EXCL", "INCL", "OBV", "PRX"],
"Polarity":  ["NEG", "POS"],
"Politeness": ["AVOID", "COL", "FOREG", "FORM", "FORM.ELEV", "FORM.HUMB", "HIGH", "HIGH.ELEV",
"HIGH.SUPR", "INFM", "LIT", "LOW", "POL"],
"Possession": ["ALN", "NALN", "PSSD", "PSSPNO+"],
"Switch-Reference": ["CN-R-MN+", "DS", "DSADV", "LOG", "OR", "SEQMA", "SIMMA", "SS", "SSADV"],
"Tense": ["1DAY", "FUT", "HOD", "IMMED", "PRS", "PST", "RCT", "RMT" ],
"Valency": ["DITR", "IMPRS", "INTR", "TR" ],
"Voice": ["ACFOC", "ACT", "AGFOC", "ANTIP", "APPL", "BFOC", "CAUS", "CFOC", "DIR", "IFOC", "INV", "LFOC",
"MID", "PASS", "PFOC", "RECP", "REFL"],
}
dimension_per_feature = {}
for f in features:
    key = [k for k,values in features_per_dimension.items()  if f in values][0]
    dimension_per_feature [f] = key
print("dimension per feature:",dimension_per_feature)

def intersect (list1,list2):
    return [l for l in list1 if l in list2]
    
features_per_dimension = [(d,intersect(f,features)) for d, f in features_per_dimension.items()]
features_per_dimension = [(d,f) for d,f in features_per_dimension if f!=[]]
features_per_dimension = dict(features_per_dimension) 


print("features per dimension:",features_per_dimension)
features_per_dimension ={
    'Animacy': ['ANIM', 'HUM', 'INAN'], 
    'Aspect': ['IPFV', 'PFV', 'PRF'], 
    'Case': ['ACC', 'DAT', 'ESS', 'GEN', 'INS', 'NOM', 'VOC'], 
    'Comparison': ['CMPR', 'SPRL'], 
    'Finiteness': ['NFIN'], 
    'Gender+': ['FEM', 'MASC', 'NEUT'], 'Mood': ['COND', 'IMP', 'IND', 'IRR'], 
    'Number': ['DU', 'PL', 'SG'], 
    'Parts of Speech': ['ADJ', 'ADV', 'N', 'V', 'V.CVB', 'V.MSDR', 'V.PTCP'], 
    'Person': ['1', '2', '3'], 'Polarity': ['NEG'], 
    'Tense': ['FUT', 'PRS', 'PST'], 
    'Voice': ['ACT', 'PASS'] 
}

low_order_hierarchy = {
    'Person': 1,
    'Number': 2,
    'Gender': 3,
    'Animacy': 4
}

low_hierarchy_of_dimension = {
    'Parts of Speech': False, #high
    'Aspect': False,# high
    'Number': True ,# low
    'Person': True,# low
    'Mood':  False,# high
    'Case':  False,# high, (in front of low)
    'Gender+':  True,# low
    'Tense':  False,# high
    'Animacy': True,# low
    'Voice':  False,# high
    'Finiteness': False, # high
    'Comparison': False, # high
    'Polarity': False, # high

}

hierarchy_per_dimension ={
    'Animacy': 6,
    'Aspect': 1,
    'Case': 2,
    'Comparison': 1, 
    'Finiteness': 1, 
    'Gender+': 5,
    'Mood': 1, 
    'Number': 4, 
    'Parts of Speech': 0, # first
    'Person': 3,
    'Polarity': 1, 
    'Tense': 1, 
    'Voice':1
}

def hierarchical_schema (features):
    split_features = features.split(";")
    POS = split_features[0]
    sorted_features = sorted(split_features, key=lambda feature: hierarchy_per_dimension[dimension_per_feature[feature]])
    low_hierarchy_features  = filter  (lambda feature:      low_hierarchy_of_dimension[dimension_per_feature[feature]], sorted_features)
    high_hierarchy_features = filter  (lambda feature: not  low_hierarchy_of_dimension[dimension_per_feature[feature]], sorted_features)
    case = [f for f in sorted_features if dimension_per_feature[f] == "Case"]
    if POS in ['V', 'V.CVB', 'V.MSDR', 'V.PTCP']:
        if case == [] or low_hierarchy_features == []:
            return ";".join(sorted_features)
        else:
            return ";".join(high_hierarchy_features) + ";NOM(" + ",".join(low_hierarchy_features) +")"
    elif POS in ['ADJ', 'ADV', 'N']:
        if case == [] or low_hierarchy_features == []:
            return ";".join(sorted_features)
        else:
            return ";".join(high_hierarchy_features) + "(" + ",".join(low_hierarchy_features) + ")"

def process_line(x):
    x = x.strip()
    lemma, target, features = x.split("\t",2)
    features = features.replace("\t",";") # features are in last or two last columns
    processed_features = hierarchical_schema(features)
    line = lemma + "\t" + processed_features + "\t" + target
    return line

path_data = "../data/"
OUT_DIR = "../../../2023InflectionST/part1/data/"
LANGS = ["dsb","slk","ces","ote","pol","csb"]
datasets = ["trn","dev","tst"]
for lang in LANGS:
    if lang == "csb": datasets = ["tst"]
    else: datasets = ["trn","dev","tst"]
    for dataset in datasets:
        with open (path_data + lang + "."+dataset,"r") as data_file, open (OUT_DIR + lang + "." + dataset,"w") as output_file:
            data_lines = data_file.readlines()
            output_file.write("\n".join([process_line(line) for line in data_lines]) + "\n")

dimension per feature: {'V': 'Parts of Speech', 'IPFV': 'Aspect', 'SG': 'Number', '2': 'Person', 'PRS': 'Tense', '3': 'Person', 'PST': 'Tense', 'PFV': 'Aspect', 'PRF': 'Aspect', '1': 'Person', 'IRR': 'Mood', 'N': 'Parts of Speech', 'ESS': 'Case', 'PL': 'Number', 'INS': 'Case', 'NOM': 'Case', 'GEN': 'Case', 'DAT': 'Case', 'ACC': 'Case', 'VOC': 'Case', 'ADJ': 'Parts of Speech', 'FEM': 'Gender+', 'NEUT': 'Gender+', 'MASC': 'Gender+', 'ANIM': 'Animacy', 'HUM': 'Animacy', 'INAN': 'Animacy', 'IMP': 'Mood', 'FUT': 'Tense', 'V.MSDR': 'Parts of Speech', 'V.PTCP': 'Parts of Speech', 'PASS': 'Voice', 'COND': 'Mood', 'V.CVB': 'Parts of Speech', 'ACT': 'Voice', 'NFIN': 'Finiteness', 'ADV': 'Parts of Speech', 'CMPR': 'Comparison', 'SPRL': 'Comparison', 'DU': 'Number', 'IND': 'Mood', 'NEG': 'Polarity'}
features per dimension: {'Animacy': ['ANIM', 'HUM', 'INAN'], 'Aspect': ['IPFV', 'PFV', 'PRF'], 'Case': ['ACC', 'DAT', 'ESS', 'GEN', 'INS', 'NOM', 'VOC'], 'Comparison': ['CMPR', 'SPRL'], 'Finiteness': [

In [ ]:

# soruce https://aclanthology.org/P15-2111.pdf
# features_per_dimension = {'Parts_of_Speech': ['V', 'N', 'ADJ', 'V.PTCP', 'V.MSDR', 'V.CVB'], # high
# 'Aspect': ['IPFV', 'PFV', 'PRF'], # high
# 'Number': ['SG', 'PL'], # low
# 'Person': ['2', '3', '1'],  # low
# 'Mood': ['IRR', 'COND', 'IMP'], # high
# 'Case': ['ESS', 'GEN', 'NOM', 'INS', 'DAT', 'VOC', 'ACC'], # high, (in front of low)
# 'Gender': ['FEM', 'MASC', 'NEUT'],  # low
# 'Tense': ['FUT', 'PRS', 'PST'], # high
# 'Animacy': ['HUM', 'ANIM', 'INAN'], # low
# 'Voice': ['PASS', 'ACT'], # high
# 'Finiteness': ['NFIN']} # high
# dimension_per_feature = {
#     'V': 'Parts_of_Speech', 'IPFV': 'Aspect', 'SG': 'Number', 'PFV': 'Aspect', 'PRF': 'Aspect', '2': 'Person', '3': 'Person', 'IRR': 'Mood', '1': 'Person', 'N': 'Parts_of_Speech', 'ESS': 'Case', 'PL': 'Number', 'GEN': 'Case', 'NOM': 'Case', 'ADJ': 'Parts_of_Speech', 'INS': 'Case', 'DAT': 'Case', 'VOC': 'Case', 'FEM': 'Gender', 'FUT': 'Tense', 'PRS': 'Tense', 'PST': 'Tense', 'COND': 'Mood', 'IMP': 'Mood', 'ACC': 'Case', 'MASC': 'Gender', 'HUM': 'Animacy', 'NEUT': 'Gender', 'V.PTCP': 'Parts_of_Speech', 'PASS': 'Voice', 'ACT': 'Voice', 'ANIM': 'Animacy', 'V.MSDR': 'Parts_of_Speech', 'INAN': 'Animacy', 'V.CVB': 'Parts_of_Speech', 'NFIN': 'Finiteness'}
# low_hierarchy_of_dimension = {
#     'Parts_of_Speech': False, #high
#     'Aspect': False,# high
#     'Number': True ,# low
#     'Person': True,# low
#     'Mood':  False,# high
#     'Case':  False,# high, (in front of low)
#     'Gender':  True,# low
#     'Tense':  False,# high
#     'Animacy': True,# low
#     'Voice':  False,# high
#     'Finiteness': False, # high
# }

# low_order_priority = {
#     'Person': 1,
#     'Number': 2,
#     'Gender': 3,
#     'Animacy': 4
# }

# order_priority = {
#     'Parts_of_Speech': 0, #high
#     'Aspect': 1,# high
#     'Number': 4 ,# low
#     'Person': 3,# low
#     'Mood':  1,# high
#     'Case':  2,# high, (in front of low)
#     'Gender':  5,# low
#     'Tense':  1,# high
#     'Animacy': 6,# low
#     'Voice':  1,# high
#     'Finiteness': 1, # high
# }

def hierarchical_schema (features):
    split_features = features.split(";")
    POS = split_features[0]
    sorted_features = sorted(split_features, key=lambda feature: order_priority[dimension_per_feature[feature]])
    low_hierarchy_features  = filter  (lambda feature:      low_hierarchy_of_dimension[dimension_per_feature[feature]], sorted_features)
    high_hierarchy_features = filter  (lambda feature: not  low_hierarchy_of_dimension[dimension_per_feature[feature]], sorted_features)
    if "V" in POS:
        return ";".join(high_hierarchy_features) + ";NOM(" + ",".join(low_hierarchy_features) +")"
    elif POS in ("N","ADJ"):
        return ";".join(high_hierarchy_features) + "(" + ",".join(low_hierarchy_features) + ")"

def process_line(x):
    x = x.strip()
    lemma, target, features = x.split("\t")
    processed_features = hierarchical_schema(features)
    line = lemma + "\t" + processed_features + "\t" + target
    return line

path_data = "data/"
OUT_DIR = "../../2023InflectionST/part1/data/"
LANGS = ["ote","pol","csb"]
datasets = ["trn","dev","tst"]
for lang in LANGS:
    if lang == "csb": datasets = ["tst"]
    else: datasets = ["trn","dev","tst"]
    for dataset in datasets:
        with open (path_data + lang + "."+dataset,"r") as data_file, open (OUT_DIR + lang + "." + dataset,"w") as output_file:
            data_lines = data_file.readlines()
            output_file.write("\n".join([process_line(line) for line in data_lines]) + "\n")